12 steps to Navier–Stokes
=====
***

This lesson complements the first interactive module of the online [CFD Python](https://github.com/barbagroup/CFDPython) class, by Prof. Lorena A. Barba, called **12 Steps to Navier–Stokes.** It was written with BU graduate student Gilbert Forsyth. It has subsequently been adapted to complement [CFDJulia](https://github.com/cysor/CFDJulia).

Array Operations with Julia
----------------

In languages such as Python, libraries like Numpy are used to handle computationaly intesive matrix operations. Numpy can increase execution speed many-times over.

In Julia, this is not the case. Computationaly intesive matrix operations can be performed natively within Julia. 

As a simple example, consider the following equation:

$$u^{n+1}_i = u^n_i-u^n_{i-1}$$

Now, given a vector $u^n = [0, 1, 2, 3, 4, 5]\ \ $   we can calculate the values of $u^{n+1}$ by iterating over the values of $u^n$ with a for loop.  

In [2]:
u = [0, 1, 2, 3, 4, 5]

for i in 2:length(u)
    println(u[i] - u[i-1])
end

1
1
1
1
1


This is the expected result and the execution time was nearly instantaneous.  If we perform the same operation as an array operation, then rather than calculate $u^n_i-u^n_{i-1}\ $ 5 separate times, we can slice the $u$ array and calculate each operation with one command:

In [3]:
u[2:end] - u[1:end-1]

5-element Array{Int64,1}:
 1
 1
 1
 1
 1

What this command says is subtract the 1st, 2nd, 3rd, 4th and 5th elements of $u$ from the 2nd, 3rd, 4th, 5th and 6th elements of $u$.

### Speed Differences

For a 6 element array, the differences between the for loop and array operation are pretty slim. There will be no appreciable difference in execution time because there are so few operations taking place. But if we revisit 2D linear convection, we can see some substantial speed differences.


In [16]:
nx = 81
ny = 81
nt = 100
c = 1
dx = 2 / (nx - 1)
dy = 2 / (ny - 1)
sigma = 0.2
dt = sigma * dx

x = range(0, stop=2, length=nx)
y = range(0, stop=2, length=ny)

u = ones(ny, nx) ##create a 1xn vector of 1's
uₙ = ones(ny, nx) 

###Assign initial conditions
u[ 0.5 .≤ y .≤ 1 , 0.5 .≤ x .≤ 1] .= 2

21×21 view(::Array{Float64,2}, [21, 22, 23, 24, 25, 26, 27, 28, 29, 30  …  32, 33, 34, 35, 36, 37, 38, 39, 40, 41], [21, 22, 23, 24, 25, 26, 27, 28, 29, 30  …  32, 33, 34, 35, 36, 37, 38, 39, 40, 41]) with eltype Float64:
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.

With our initial conditions all set up, let's first try running our original nested loop code, making use of BenchmarkTools.jl, which will help us evaluate the performance of our code. 

**Note**: The `@benchmark` macro provided by BenchmarkTools.jl will run the code several times and then give  min, max and mean execution times, along with estimatied memory consumption. The code that we want to benchmark must be placed inside a function and the variables required by the function should be included in the function definition or passed as parameters.

The execution times below will vary from machine to machine.  Don't expect your times to match these times, but you _should_ expect to see the same general trend in decreasing execution time as we switch to array operations.

In [18]:
using BenchmarkTools

In [17]:
function ForLoopMethod()
    nx = 81
    ny = 81
    nt = 100
    c  = 1
    Δx = 2 / (nx - 1)
    Δy = 2 / (ny - 1)
    σ  = 0.2
    Δt = σ * Δx

    x = range(0, stop=2, length=nx)
    y = range(0, stop=2, length=ny)

    u = ones(ny, nx) ##create a 1xn vector of 1's

    u[0.5 .≤ y .≤ 1 , 0.5 .≤ x .≤ 1] .= 2

    uⁿ⁺¹ = u # uⁿ⁺¹ points to the values stored by u, i.e it provides a new name to refer to u with
    
    for n in 1:nt + 1 ##loop across number of time steps
        uⁿ = copy(uⁿ⁺¹)
        row, col = size(uⁿ⁺¹)
        for j ∈ 1:row
            for i ∈ 1:col
                # Implement boundary conditions using conditional (if/else) statements
                if j == 1
                    uⁿ⁺¹[j,i] = 1.0
                elseif j == row
                    uⁿ⁺¹[j,i] = 1.0
                elseif i == col
                    uⁿ⁺¹[j,i] = 1.0
                elseif i == 1
                    uⁿ⁺¹[j,i] = 1.0
                else
                    uⁿ⁺¹[j,i] = (uⁿ[j, i] - 
                                (c * Δt / Δx * (uⁿ[j, i] - uⁿ[j, i - 1])) - 
                                (c * Δt / Δy * (uⁿ[j, i] - uⁿ[j - 1, i])))
                end
            end
        end
    end
    return uⁿ⁺¹
end
@benchmark ForLoopMethod()

BenchmarkTools.Trial: 
  memory estimate:  5.12 MiB
  allocs estimate:  217
  --------------
  minimum time:     3.724 ms (0.00% GC)
  median time:      9.950 ms (0.00% GC)
  mean time:        9.642 ms (7.07% GC)
  maximum time:     25.235 ms (44.95% GC)
  --------------
  samples:          518
  evals/sample:     1

With the for loop implementation above, the mean execution time achieved was 9.6 milli seconds (on an i7-7700HQ). Memory usage was 5.12 MiB and total memory allocations was 217. For all these metrics lower is better. Let's compare that with the performance of the same code implemented with array operations:

In [21]:
function MatrixMethod()
    nx = 81
    ny = 81
    nt = 100
    c  = 1
    Δx = 2 / (nx - 1)
    Δy = 2 / (ny - 1)
    σ  = 0.2
    Δt = σ * Δx

    x = range(0, stop=2, length=nx)
    y = range(0, stop=2, length=ny)

    u = ones(ny, nx) ##create a 1xn vector of 1's

    u[0.5 .≤ y .≤ 1 , 0.5 .≤ x .≤ 1] .= 2

    uⁿ⁺¹ = u # uⁿ⁺¹ points to the values stored by u, i.e it provides a new name to refer to u with
    
    for n in 1:nt + 1 ##loop across number of time steps
        uⁿ = copy(uⁿ⁺¹)
        
        u[2:end, 2:end] = (uⁿ[2:end, 2:end] - (c * Δt / Δx * (uⁿ[2:end, 2:end] - uⁿ[2:end, 1:end-1])) - 
                                    (c * Δt / Δy * (uⁿ[2:end, 2:end] - uⁿ[1:end-1, 2:end])))
        u[1, :]   .= 1 # Dot broadcasting is used to set each value in the matrix slice equall to 1.
        u[end, :] .= 1 # This syntax should be familar for people who have used MATLAB. 
        u[:, 1]   .= 1
        u[:, end] .= 1
    end
    return u
end
@benchmark MatrixMethod()

BenchmarkTools.Trial: 
  memory estimate:  59.45 MiB
  allocs estimate:  2439
  --------------
  minimum time:     15.693 ms (16.34% GC)
  median time:      39.639 ms (17.17% GC)
  mean time:        39.903 ms (17.11% GC)
  maximum time:     119.139 ms (22.50% GC)
  --------------
  samples:          126
  evals/sample:     1

As you can see, the speed decrease and memory usage increase is substantial. The same calculation goes from 9.6 milliseconds to 40 milliseconds. Similiarly memory allocations and total memory usage increase significantly, from 5.12 MiB, 217 allocations to 59.45MiB, 2439 allocation. 

Although the absolute memory usage and execution time are low these speed gains will increase exponentially with the size and complexity of the problem being evaluated. Try running the benchmarks again with nx and ny equall to 1001 and nt = 1000, note this may take a long time to run depending on your hardware.

This behaviour in julia is unlike languages such as MATLAB where vectorised/array operations are faster than for loop iteration. MATLAB is able to do this due to a series of optimisations. The use of for loops to perform matrix operations and if/else statements to implement boundary conditions are a much better implementation from a computer science persepective. The Julia documentation provides a comprehensive list of [performance tips](https://docs.julialang.org/en/v1/manual/performance-tips/index.html) that can help improve the performance of Julia programs.